In [0]:
# 標準出力
print('{}'.format('hello, world'))

In [0]:
# 関数
def add(x, y):
    print(x + y)

add(3, 5)

In [0]:
# データの取得
import keras
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [0]:
train_images_size = len(train_images)
test_images_size = len(test_images)
print('訓練データ数 => {}'.format(train_images_size))
print('テストデータ数 => {}'.format(test_images_size))

image_size = train_images[0].size
image_height = train_images[0].shape[0]
image_width = train_images[0].shape[1]
print('画像のサイズ => {}\n縦の大きさ => {}\n横の大きさ => {}'.format(image_size, image_width, image_height))

In [0]:
# 0番目のデータを表示する
# print(train_images[0])

In [0]:
import numpy as np

# 正規化したとき小数で表現できるようにfloatに変換
train_images = train_images.astype(np.float)

# 明るさを0～1に正規化する
train_images = np.multiply(train_images, 1.0 / 255.0)

In [0]:
# 画像の可視化
def display(img):    
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    one_image = img.reshape(image_width, image_height)
    plt.imshow(one_image, cmap=cm.binary)

# 表示する画像を指定する
IMAGE_TO_DISPLAY = 20

# ラベルの文字
print(train_labels[IMAGE_TO_DISPLAY])
display(train_images[IMAGE_TO_DISPLAY])

In [0]:
# 画像を1次元配列に変換
train_images = train_images.reshape([train_images_size, image_size])
print(train_images[IMAGE_TO_DISPLAY].size)

In [0]:
labels_count = np.unique(train_labels).shape[0]
print('ラベルの種類 => {}'.format(labels_count))

In [0]:
# 検証データに回す量
VALIDATION_SIZE = 2000

# ランダムに並べ替える
# シードを同じにすることで画像とラベルの対応が崩れないようにしている
rand = np.random.randint(0, 9)
np.random.seed(rand)
np.random.shuffle(train_images)
np.random.seed(rand)
np.random.shuffle(train_labels)

In [0]:
# 訓練データと検証データに分離
validation_images = train_images[:VALIDATION_SIZE]
validation_labels = train_labels[:VALIDATION_SIZE]

train_images = train_images[VALIDATION_SIZE:]
train_labels = train_labels[VALIDATION_SIZE:]

train_images_size = train_images.shape[0]
validation_images_size = validation_images.shape[0]

print('訓練データ数 => {}\n検証データ数 => {}'.format(train_images_size, validation_images_size))

In [0]:
# One-hotベクトルを作る
from keras.utils import np_utils
train_labels = np_utils.to_categorical(train_labels).astype(np.uint8)
print(train_labels[IMAGE_TO_DISPLAY])

In [0]:
import tensorflow as tf

# 定数の例
c = tf.constant('hello, world')
with tf.Session() as sess:
    print(sess.run(c))

In [0]:
# 変数の例
def calc():
    w = tf.Variable(10)
    # （Variableで警告が出るが気にしないで進める）
    x = tf.constant(5)
    return tf.assign(w, w + x)
  
with tf.Session() as sess:
    c = calc()
    # 変数を使う場合は定義後、実行前に初期化が必要
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(c))

In [0]:
# プレースホルダの例
x = tf.placeholder(tf.float32)
y = tf.square(x)

with tf.Session() as sess:
    print(sess.run(y, feed_dict={x: 20}))
# どちらでも実行できる
sess = tf.InteractiveSession()
print(sess.run(y, feed_dict={x: 20}))
sess.close()

In [0]:
# 畳込み
def conv2d(x, W):
    # x: 入力（テンソル）
    # W: フィルタ
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# プーリング
def max_pool_2x2(x):
    # x: 入力
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
# 重み
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# バイアス
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [0]:
# 画像のプレースホルダ
x = tf.placeholder('float', shape=[None, image_size])
# ラベルのプレースホルダ
y_ = tf.placeholder('float', shape=[None, labels_count])

In [0]:
# 画像の次元を変換する（入力層）
image = tf.reshape(x, [-1,image_width , image_height,1])

# 1つ目の畳み込み層
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# 畳込みして活性化関数に通す
h_conv1 = tf.nn.relu(conv2d(image, W_conv1) + b_conv1)
# 最大プーリング
h_pool1 = max_pool_2x2(h_conv1)

In [0]:
# 2つ目の畳み込み層
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [0]:
# 画像の次元を変換する
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])

# 全結合層
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [0]:
# ドロップアウト
rate = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, rate=rate)

In [0]:
# 出力層
W_fc2 = weight_variable([1024, labels_count])
b_fc2 = bias_variable([labels_count])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [0]:
# 損失関数
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

# 最適化関数
LEARNING_RATE = 1e-4
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)

In [0]:
# 評価

# [0.05, 0.01, 0.04, 0.1, 0.02, 0.05, 0.2, 0.03, 0.4, 0.1] => 8
predict = tf.argmax(y,1)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [0]:
# エポックの回数
epochs_completed = 0
index_in_epoch = 0
# 訓練データ数
num_examples = train_images.shape[0]

# バッチごとのデータを返す
def next_batch(batch_size):
    
    global train_images
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # すべての訓練データを使い切ったらランダムに並べ替えて再利用する
    if index_in_epoch > num_examples:
        epochs_completed += 1
        # データをシャッフルする
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_images[perm]
        train_labels = train_labels[perm]
        # 次のエポックを始める
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_images[start:end], train_labels[start:end]

In [0]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

In [0]:
TRAINING_ITERATIONS = 2500
BATCH_SIZE = 50
DROPOUT = 0.5

for i in range(TRAINING_ITERATIONS):
    # 新しいバッチを取得する
    batch_xs, batch_ys = next_batch(BATCH_SIZE)
    # バッチを訓練する
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, rate: DROPOUT})

In [0]:
# 検証データのラベルのOne-hotベクトルを作る
onehot_validation_labels = np_utils.to_categorical(validation_labels).astype(np.uint8)

# 検証データの精度を評価する
validation_accuracy = accuracy.eval(feed_dict={x: validation_images, 
                                               y_: onehot_validation_labels, 
                                               rate: 0.0})
print('検証データの精度 => %.4f'%validation_accuracy)

In [0]:
# 検証データで予測する
predicted_lables = np.zeros(validation_images.shape[0])
predicted_lables = predict.eval(feed_dict={x: validation_images, rate: 0.0})
print('検証データ数 => {}'.format(len(predicted_lables)))

In [0]:
# 検証データの画像と予測結果を出力する
IMAGE_TO_DISPLAY = 20

print('正解ラベル => {}\n予測したラベル => {}'.format(validation_labels[IMAGE_TO_DISPLAY], predicted_lables[IMAGE_TO_DISPLAY]))
display(validation_images[IMAGE_TO_DISPLAY])

In [0]:
# 正解と予測が異なるデータを見る
np.where(validation_labels != predicted_lables)

In [0]:
# テストデータを読み込む
test_images = test_images.astype(np.float)
test_images = np.multiply(test_images, 1.0 / 255.0)
test_images = test_images.reshape([test_images_size, image_size])
test_labels = np_utils.to_categorical(test_labels).astype(np.uint8)
print('テストデータの画像 => {0[0]},{0[1]}\nテストデータのラベル => {1[0]},{1[1]}'.format(test_images.shape, test_labels.shape))

In [0]:
test_accuracy = accuracy.eval(feed_dict={x: test_images, y_: test_labels, rate: 0.0})
print('テストデータの精度 => %.4f'%test_accuracy)

In [0]:
sess.close()